<a href="https://colab.research.google.com/github/duanchi1230/NLP_Project_AI2_Reasoning_Challenge/blob/arc-chi/project_arc_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [3]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

Cloning into 'bert_repo'...
remote: Enumerating objects: 340, done.
remote: Total 340 (delta 0), reused 0 (delta 0), pack-reused 340
Receiving objects: 100% (340/340), 300.28 KiB | 12.51 MiB/s, done.
Resolving deltas: 100% (185/185), done.


In [0]:
import datetime
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization
# import tfhub 
import tensorflow_hub as hub
import json
import tensorflow as tf

In [0]:

def DataProcessor(path):
  example = []
  for line in open(path):
    line = json.loads(line)
    for choice in line["question"]["choices"]:
      if choice["label"] == line["answerKey"]:
        example.append(run_classifier.InputExample(line["id"], line["question"]["stem"], choice["text"], "1"))
      if choice["label"] != line["answerKey"]:
        example.append(run_classifier.InputExample(line["id"], line["question"]["stem"], choice["text"], "0"))
  return example

In [0]:
arc_corpus = open("/content/drive/My Drive/CSE576_NLP/Data/ARC_Corpus.txt","r+")

arc_easy_dev = DataProcessor("/content/drive/My Drive/CSE576_NLP/Data/ARC-Easy/ARC-Easy-Dev.jsonl")
arc_easy_test = DataProcessor("/content/drive/My Drive/CSE576_NLP/Data/ARC-Easy/ARC-Easy-Test.jsonl")
arc_easy_train = DataProcessor("/content/drive/My Drive/CSE576_NLP/Data/ARC-Easy/ARC-Easy-Train.jsonl")

# for train_examples in arc_easy_dev:
#   print(train_examples.guid) 
#   print(train_examples.text_a) 
#   print(train_examples.text_b) 
#   print(train_examples.label)


In [79]:
len(arc_easy_test)

9501

In [30]:
OUTPUT_DIR = '/content/drive/My Drive/CSE576_NLP/project-arc-code'
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

In [0]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500
label_list = ["0", "1"]

num_train_steps = int(len(arc_easy_train) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [41]:
bert_config = modeling.BertConfig.from_json_file('/content/drive/My Drive/CSE576_NLP/'+BERT_MODEL+'/bert_config.json')
init_checkpoint = '/content/drive/My Drive/CSE576_NLP/'+BERT_MODEL+'/bert_model.ckpt'

run_config = tf.contrib.tpu.RunConfig(
      model_dir='/content/drive/My Drive/CSE576_NLP',
      save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
)

model_fn = run_classifier.model_fn_builder(
    bert_config=bert_config,
    num_labels=len(label_list),
    init_checkpoint=init_checkpoint,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False,
    use_one_hot_embeddings=False)

estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=False,
      eval_on_tpu =False,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=TRAIN_BATCH_SIZE,
      eval_batch_size=EVAL_BATCH_SIZE,
      predict_batch_size=PREDICT_BATCH_SIZE)

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/CSE576_NLP', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fee5cf36e80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=None, num_cores

INFO:tensorflow:Using config: {'_model_dir': '/content/drive/My Drive/CSE576_NLP', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fee5cf36e80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=2, num_shards=None, num_cores

INFO:tensorflow:_TPUContext: eval_on_tpu False


INFO:tensorflow:_TPUContext: eval_on_tpu False


In [0]:
# Train the model
def model_train(estimator, data_set, label_list, tokenizer, TRAIN_BATCH_SIZE, MAX_SEQ_LENGTH, num_train_steps):
  print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      data_set, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(data_set)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))

In [57]:
model_train(estimator, arc_easy_train, label_list, tokenizer, TRAIN_BATCH_SIZE, MAX_SEQ_LENGTH, num_train_steps)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...
INFO:tensorflow:Writing example 0 of 9002


INFO:tensorflow:Writing example 0 of 9002


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: Mercury_7220990


...

INFO:tensorflow:Loss for final step: 0.60158706.


INFO:tensorflow:Loss for final step: 0.60158706.


INFO:tensorflow:training_loop marked as finished


INFO:tensorflow:training_loop marked as finished


***** Finished training at 2020-04-06 09:25:35.127049 *****


In [0]:
def model_eval(estimator, data_set, label_list, tokenizer, EVAL_BATCH_SIZE, MAX_SEQ_LENGTH, num_train_steps):
  # Eval the model.
  eval_features = run_classifier.convert_examples_to_features(
      data_set, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(data_set)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(data_set) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))

  return result
  # output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  # with tf.gfile.GFile(output_eval_file, "w") as writer:
  #   print("***** Eval results *****")
  #   for key in sorted(result.keys()):
  #     print('  {} = {}'.format(key, str(result[key])))
  #     writer.write("%s = %s\n" % (key, str(result[key])))

In [85]:
eval_result = model_eval(estimator, arc_easy_dev, label_list, tokenizer, EVAL_BATCH_SIZE, MAX_SEQ_LENGTH, num_train_steps)

INFO:tensorflow:Writing example 0 of 2281


INFO:tensorflow:Writing example 0 of 2281


INFO:tensorflow:*** Example ***


INFO:tensorflow:*** Example ***


INFO:tensorflow:guid: MCAS_2000_4_6


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 843: /content/drive/My Drive/CSE576_NLP/model.ckpt-843


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 843: /content/drive/My Drive/CSE576_NLP/model.ckpt-843


INFO:tensorflow:evaluation_loop marked as finished


INFO:tensorflow:evaluation_loop marked as finished


***** Finished evaluation at 2020-04-06 09:52:23.618731 *****


In [86]:
eval_result

{'eval_accuracy': 0.7491228,
 'eval_loss': 0.5333774,
 'global_step': 843,
 'loss': 0.5333774}